In [1]:
import numpy as np
import requests
import pandas as pd
import scipy
import json
import urllib
import os

In [2]:
musicBrainzURL =  "https://musicbrainz.org/ws/2/"
transcription_data = pd.read_csv("../BillboardTranscriptionData.tsv", sep="\t")
sample_data = pd.read_csv("../BillboardSampleData.tsv", sep="\t")

In [53]:
path = "../Humdrum/MelodicTranscriptions/"
ext = ".krn"
files = os.listdir(path)
file_names = [os.path.splitext(file)[0] for file in files if os.path.splitext(file)[1] == ext]
file_names.sort()

In [54]:
len(file_names)

214

In [56]:
def lookup_metadata(track_name: str, artist_name: str, incs: str) -> dict:
    """Looks up the metadata for a listen using track name and artist name."""
    params = {
        "recording_name": track_name,
        "artist_name": artist_name,
        "type": "single"
    }
    if incs:
        params["metadata"] = True
        params["incs"] = incs
#         params["type"] = "release"
    response = requests.get(
        url="https://api.listenbrainz.org/1/metadata/lookup/",
        params=params
    )
    response.raise_for_status()
    return response.json()

In [57]:
lookups = transcription_data[transcription_data["FileName"].isin(file_names)]
lookups["mbid"] = [lookup_metadata(title, artist, "").get("recording_mbid") for title, artist in zip(lookups["TITLE"], lookups["ARTIST"])]

<ipython-input-57-fb18fc6dbdc5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lookups["mbid"] = [lookup_metadata(title, artist, "").get("recording_mbid") for title, artist in zip(lookups["TITLE"], lookups["ARTIST"])]


In [58]:
len(lookups)

214

In [59]:
for i in range(len(lookups)):
    row = lookups.iloc[i]
    humfile = open(path + row["FileName"] + ext, "a")
    humfile.write("!!!MBID: "+str(row["mbid"])+"\n")
    humfile.close()